# pyBREP Description and to do list

**Dear soon-to-be pyBREP enthusiast!**

This notebook is supposed to help you get started with, use and further develope pyBREP. It contains
- A quick overview over the structure
- A documentation of the material (some of it is developmental history, some of it is for testing)
- Differences to the original (scheme) BREP
- A to-do list containing known caveats and more nice-to-have things (and, if available, my initial ideas on how to tackle them)

If you have any question, do not hesitate to contact me, I am happy to try and help! 

Best, Ines

## Short overview over the structure of pyBREP

pyBREP consists essentially of the following components:
- **The population files:** So far there are only Golgi and Granule cell populations. Those populations contain routines to read in data from coordinate files, render cell structures (like dendrites or axons) by representing them with points or lines, to save those cell structures and to prepare so-called query points.
- **Query_point** is a small class that contains the coordinates of the data points from one population, as well as other information (for example, each Golgi cell has a lot of points that represent the dendrites, so each point has to be labeled in order to associate it to one particular segment of one particular dendrite.)
- **Connector_* classes:** Those classes take in two sets of data (either two Query_point objects or two arrays which are upon import converted into Query_point objects) and connects them. It usually does so by using functions defined in the **parallel_utils** file. By implementing similar classes or parallel_utils functions, new connection methods could be implemented. 

## Overview over the notebooks that were used for the development:

The following notebooks should be available somewhere, in case there is anything that has to be rechecked where a development history would be helpful to resolve any issues:
**pyBREP_first:** The ancient development history
This is the oldest notebook. It contains:
- A first overview over the files that were generated by the scheme BREP
- The initial versions of most functions of the actual pyBREP files as well as methods to further analyse the original BREP files in order to reproduce them (e.g. additional visualization methods)
- Some additional functionalities that were in the end not needed (e.g. the Golgi-Golgi gap junctions)
- Different versions of the KDTree search algorithm that were tested against each other for speed
- Some additional data processing methods (e.g. subsampling methods) that were used for the development process.
- Some theoretical insights on various things. 

**Golgi_cell_checkout:** About Golgi cells 
This notebook has an old and a rather new part. The old part contains:
- Functions to look at and visualize the Golgi cell data produced by BREP
- Functions that generate and store Golgi cell data similar to the BREP data.
(-> basically, it is at the same stage as the pyBREP_first notebook with this, but only treating the Golgi cell part)
The new part rechecks a few of those things that were erroneous in the first solution, however as the pyBREP.py file already existed at this stage, it only calls methods from in there.
Especially in the old part should at some stages be redundant with the pyBREP_first code, but probably the version in this notebook is the more important one.

**BREP_original_output_evaluation:**
This notebook was used to investigate in the files that the original BREP program produces. 
In the first part, data that has been used in the original simulation is investigated.
The second part deals more with smaller aspects and data that was used by running the scheme BREP locally and on a smaller dataset.
Related investigations have been done in other files as well:
- In the Golgi_cell_checkout notebook, the rendering of Golgi cells is analysed and reproduced, and some analysiy on the Golgi-Golgi connections is performed
- The BREPpy_first notebook contains several investigations, mostly with a focus of single connections

**Test_Code_Notebook**
The goal of this notebook is to test the code by visualizing connections and connectivity statistics.
The connectivity part can deal with the whole simulation, the second part runs better on scaled-down versions of the simulation. 
Some of the routines have predecessor routines in the BREP_Original_Output code, so results can be compared with the ones from the original BREP code.


## Things that should or could be done for the code

**To do: (rather urgent): Population generation**

Right now, the coordinates of the populations are generated from the simulation and pyBrep just reads them in.
However, it seems as if those coordinates are 'not random enough': If you plot them as projections to the x-y or the y-z axis, you can see that there are 'stripes' in the z axis. With pyBrep, this leads to 



**To do (more relaxed):  Multiple connections**

Especially in the case aa to Golgi, the spacing between the points on the Golgi cell dendrites is considerably smaller than the radius around the aa in which points are searched. Thus, it often occurs that one connection is found several times. This has happened both in the old and the new program. 
I spoke to Erik about that, and as far as I got him, the best would be to summarize multiple connections in one synapse with a higher strength, as it does indeed occur that there are several adjacent connections, but the way their input is summarized this implementation is better. 
One idea to do that within the simulation would be to chunk the data for the workers so that all points belonging to one cell are assigned to the same worker and then performing this check in the parallel_utils file for each worker. 
Alternatively, it might be the easiest to perform this check at the read-in to the big simulation.


**To do (more relaxed): Self-Connections**

Right now, if a cell population has connections within, it is possible that a cell connects to itself (e.g. in the case of Golgi-Golgi connections.)
If necessary, this could probably be solved easiest by adding a check to the parallel_util function, or by checking at the import of the connections to the big simulation (similar to the multiple connections above)


**General nice-to-have: More testing, also with the simulation**  

Check for robustness using slightly different parameters for the cell description, a different chunk of the membrane (preferably extend in the PF direction), different densities, ...


**Possible extension: Test and debug the code with more dendrites**

Sungho mentioned that he possibly would like to have more dendrites on the Golgi cells. I generally tried to make it possible that there is an arbitrary number of dendrites for the Golgi cells. But right now there is no specification on the angles that those dendrites would have, so a solution would have to be found for that (equal spacing?). So I did not evaluate whether it works, and very likely it does not do so from the beginning.


**Possible extension: Test varying aa length (talk to Sungho, that is probably more a new project...)**

There is already a method for at least rendering the cells implemented in the Granule_pop file (works only with the Connect_2D class though.) However, much more has not been done (no testing etc.)

## Things that changed with respect to the scheme BREP

### Input/Output:
- It does not matter any more whether sorted or not sorted coordinate files are used. The saving functions right now have the .sorted. bit in their name, but they only write out the coordinates they got as input/they generated themselves randomly. 
- _distance.dat files were previously the distance a signal would have to travel from one soma to the other. Now it is the distance that the signal makes from the soma to the connection. (I.e. for the PF, the length of the AA is added). 
- BREP uses global indices, i.e. the Golgi cells have IDs 1-1995, the Granule cells 1996-end. pyBREP right now uses IDs always going from 0-end. 


### In the parameters.hoc file...
- PFzoffset: this parameter was supposed to be the AA length, but it is not read in. It has now been replaced by the sum of depths of the Granule cell layer and the pyramidal cell layer. This was already done so in the scheme BREP. The PFzoffset parameter is thus unused.
- Dendrite_theta_min/max: In the scheme BREP and pyBREP, they are both just the expected values of the angle of the direction of the dendrite. If more than two dendrites are used, the direction of them would have to be specified and implemented accordingly.
- ** check github

## Some final remarks...
For future connection problems that need more parameters, check and data processing, I think the easiest way to extend would be as follows:
- Use population files to assign additional cell information to the Query_point object (maybe give the latter one a \**kwargs option and just store all data that is passed with it)
- Use a new Connector_* file, or possibly even just add a passable function that refers to the parallel_util file to implement the different processing steps.

